## Exercises

### Data Acquisition

These exercises should go in a notebook or script named `wrangle`. Add, commit, and push your changes.

This exercises uses the `case.csv`, `dept.csv`, and `source.csv` files from the
san antonio 311 call dataset.

1. Read the case, department, and source data into their own spark dataframes.

1. Let's see how writing to the local disk works in spark:

    - Write the code necessary to store the source data in both csv and json
      format, store these as `sources_csv` and `sources_json`
    - Inspect your folder structure. What do you notice?

9. Inspect the data in your dataframes. Are the data types appropriate? Write
   the code necessary to cast the values to the appropriate types.


---

1. How old is the latest (in terms of days past SLA) currently open issue? How
   long has the oldest (in terms of days since opened) currently opened issue
   been open?
1. How many Stray Animal cases are there?
1. How many service requests that are assigned to the Field Operations
   department (`dept_division`) are not classified as "Officer Standby" request
   type (`service_request_type`)?

1. Convert the `council_district` column to a string column.
1. Extract the year from the `case_closed_date` column.
1. Convert `num_days_late` from days to hours in new columns `num_hours_late`.

1. Join the case data with the source and department data.
1. Are there any cases that do not have a request source?

1. What are the top 10 service request types in terms of number of requests?
1. What are the top 10 service request types in terms of average days late?
1. Does number of days late depend on department?
1. How do number of days late depend on department and request type?

In [1]:
import pyspark
#creates the instance of spark and connects to the java virt machine, how we call the spark methods
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [5]:
import pandas as pd
import numpy as np
from pydataset import data
from pyspark.sql.functions import *
from pyspark.sql.functions import col, expr
# Note: The pyspark avg and mean functions are aliases of eachother
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean
from pyspark.sql.functions import lit
from pyspark.sql.functions import regexp_extract, regexp_replace
from pyspark.sql.functions import when
from pyspark.sql.functions import asc, desc
from pyspark.sql.functions import month, year, quarter

### data prep

In [3]:
df = case = spark.read.csv('data/case.csv', header=True, inferSchema=True)
print('nrows:', df.count())
stray_animal_cases = df.filter(df.service_request_type == 'Stray Animal').count()
print('stray animal cases:', stray_animal_cases)

nrows: 841704
stray animal cases: 26760


In [6]:
# Rename column
df = df.withColumnRenamed('SLA_due_date', 'case_due_date')

# Convert to better data types
df = (
    df.withColumn('case_late', col('case_late') == 'YES')
    .withColumn('case_closed', col('case_closed') == 'YES')
)
df = df.withColumn('council_district', format_string('%04d', col('council_district')))
df = (
    df.withColumn('case_opened_date', to_timestamp(col('case_opened_date'), 'M/d/yy H:mm'))
    .withColumn('case_closed_date', to_timestamp(col('case_closed_date'), 'M/d/yy H:mm'))
    .withColumn('case_due_date', to_timestamp(col('case_due_date'), 'M/d/yy H:mm'))
)

# Cleanup text data
df = df.withColumn('request_address', lower(trim(col('request_address'))))
# Extract zipcode
df = df.withColumn('zipcode', regexp_extract(col('request_address'), r'\d+$', 0))

# Create a `case_lifetime` feature
df = (
    df.withColumn('case_age', datediff(current_timestamp(), 'case_opened_date'))
    .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
    .drop('case_age', 'days_to_closed')
)

# Join departments and sources
depts = spark.read.csv('data/dept.csv', header=True, inferSchema=True)
sources = spark.read.csv('data/source.csv', header=True, inferSchema=True)

df = df.join(depts, 'dept_division', 'left').join(sources, 'source_id', 'left')

# # Train Test Split
# train, test = df.randomSplit([.8, .2], seed=123)
# train, validate, test = df.randomSplit([.7, .15, .15], seed=123)

In [7]:
df.count()

855269

In [8]:
#if i want to query dataframe with Spark SQL:
df.createOrReplaceTempView('df')

In [9]:
# How old is the latest (in terms of days past SLA) currently open issue?
# How long has the oldest (in terms of days since opened) currently opened issue been open?
spark.sql('''
SELECT DATEDIFF(current_timestamp, case_due_date) AS days_past_due
FROM df
WHERE NOT case_closed
ORDER BY days_past_due DESC
LIMIT 15
''').show()

+-------------+
|days_past_due|
+-------------+
|         2067|
|         2067|
|         2067|
|         2066|
|         2064|
|         2060|
|         2060|
|         2059|
|         2058|
|         2058|
|         2054|
|         2053|
|         2053|
|         2053|
|         2050|
+-------------+



In [10]:
# How many Stray Animal cases are there?
df.filter(df.service_request_type == 'Stray Animal').count()

27361

In [11]:
(
    df.groupBy('service_request_type')
    .count()
    .filter(expr('service_request_type == "Stray Animal"'))
    .show()
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|27361|
+--------------------+-----+



In [12]:
# How many service requests that are assigned to the Field Operations department (dept_division)
# are not classified as "Officer Standby" request type (service_request_type)?
(
    df.filter(df.dept_division == 'Field Operations')
    .filter(df.service_request_type != 'Officer Standby')
    .count()
)

116295

In [13]:
# Another way to do it
(
    df.filter(expr("dept_division == 'Field Operations'"))
    .filter(expr('service_request_type != "Officer Standby"'))
    .count()
)

116295

In [14]:
# Convert the council_district column to a string column.

# Already done in the data prep

In [15]:
# Extract the year from the case_closed_date column.
df.select('case_closed_date', year('case_closed_date')).show(5)

+-------------------+----------------------+
|   case_closed_date|year(case_closed_date)|
+-------------------+----------------------+
|2018-01-01 12:29:00|                  2018|
|2018-01-03 08:11:00|                  2018|
|2018-01-02 07:57:00|                  2018|
|2018-01-02 08:13:00|                  2018|
|2018-01-01 13:29:00|                  2018|
+-------------------+----------------------+
only showing top 5 rows



In [16]:
# Convert num_days_late from days to hours in new columns num_hours_late.
(
    df.withColumn('num_hours_late', df.num_days_late * 24)
    .select('num_days_late', 'num_hours_late')
    .show()
)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
|       -29.74398148| -713.8555555199999|
|       -14.70673611|      -352.96166664|
|       -14.70662037|      -352.95888888|
|       -14.70662037|      -352.95888888|
|       -14.70649306|      -352.95583344|
|       -14.70649306|      -352.95583344|
|       -14.70636574|      -352.95277776|
|          -14.70625|-352.95000000000005|
|       -14.70636574|      -352.95277776|
|       -14.70623843|-352.94972232000003|
|-14.705891199999998|-352.94138879999997|
|       -14.70600694|      -352.94416656|
|       -14.70576389|      -352.93833336|
|       -14.70576389|      -352.93833336|
|       -14.70564815|       -352.9355556|
+-------------------+-------------

In [17]:
# Join the case data with the source and department data.

# already joined in the data prep

In [18]:
sources.show()

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows



In [19]:
sources.sort(col('source_id').desc()).show()

+---------+-----------------+
|source_id|  source_username|
+---------+-----------------+
|  yh24110| Yojani Hernandez|
|  yc16753|    Yvonne Casias|
|  vk26526|    Vincent Kosub|
|  vb22265| Vanessa Burciaga|
|  ts15690|  Thelma Martinez|
|  sw26367|Samantha Wickwire|
| svcCRMSS|         svcCRMSS|
| svcCRMLS|         svcCRMLS|
| svcCFlag|         CityFlag|
|  sv24848|  Salvador Valdez|
|  ss26317|     Sakib Shaikh|
|  ss21394|  Shirley Sanchez|
|  ss09159|   Sylvia Sanchez|
|  sp26368|  Steven Peterson|
|  sg26196|   Samantha Garza|
|  sg22264|  Steven Martinez|
|  ru26699|      Raul Ubides|
|  rs16746|Ronald S. Sorrell|
|  rb05270|       Rene Bomar|
|  ps01944|   Paula Stallcup|
+---------+-----------------+
only showing top 20 rows



In [20]:
(
    case.join(sources, "source_id", "left")
    .sort(col("source_username"))
    .show(5, vertical=True)
)

-RECORD 0------------------------------------
 source_id            | af26445              
 case_id              | 1014219925           
 case_opened_date     | 2/5/18 14:16         
 case_closed_date     | 2/7/18 12:16         
 SLA_due_date         | 5/1/18 14:16         
 case_late            | NO                   
 num_days_late        | -83.08341435         
 case_closed          | YES                  
 dept_division        | Signals              
 service_request_type | Signal Timing Mod... 
 SLA_days             | 85.0                 
 case_status          | Closed               
 request_address      | EWING HALSELL and... 
 council_district     | 8                    
 source_username      | Alex Franklin        
-RECORD 1------------------------------------
 source_id            | af26445              
 case_id              | 1014220627           
 case_opened_date     | 2/5/18 16:17         
 case_closed_date     | 2/6/18 10:03         
 SLA_due_date         | 4/9/18 16:

In [21]:
# Are there any cases that do not have a request source?
# are there any null values for source_id?
(
    df.select(df.source_id.isNull().cast('int').alias('is_null'))
    .agg(sum('is_null'))
    .show()
)

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



In [22]:
df.filter(col('source_id').isNull()).show(vertical=True)

(0 rows)



In [23]:
# What are the top 10 service request types in terms of number of requests?
(
    df.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



In [24]:
# What are the top 10 service request types in terms of average days late?
# - just the late cases
# - for the late cases:
#   - what is the average number of days late by request type?
(
    df.where('case_late') # just the rows where case_late == true
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.2051760849428 |262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942624|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



In [25]:
# Does number of days late depend on department?
(
    df.filter('case_late')
    .groupby('dept_name')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

+-------------------------+---------+------------+
|dept_name                |days_late|n_cases_late|
+-------------------------+---------+------------+
|Metro Health             |6.5      |854         |
|Solid Waste Management   |7.1      |33729       |
|Trans & Cap Improvements |10.7     |5529        |
|Parks and Recreation     |22.4     |3810        |
|Animal Care Services     |23.4     |23751       |
|Code Enforcement Services|48.1     |25467       |
|Development Services     |67.2     |840         |
|Customer Service         |88.2     |2035        |
|null                     |210.9    |132         |
+-------------------------+---------+------------+



In [26]:
df.groupby('dept_name').count().show(truncate=False)

+-------------------------+------+
|dept_name                |count |
+-------------------------+------+
|Animal Care Services     |119362|
|null                     |198   |
|Solid Waste Management   |286287|
|Development Services     |1397  |
|Trans & Cap Improvements |97841 |
|Customer Service         |2889  |
|Metro Health             |5313  |
|Parks and Recreation     |19964 |
|Code Enforcement Services|321984|
|City Council             |34    |
+-------------------------+------+



In [27]:
# How do number of days late depend on department and request type?
(
    df.filter("case_closed")
#     .filter("case_late")
    .groupby("standardized_dept_name", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .sort(asc("days_late"))
    .show(40, truncate=False)
)

+------------------------+--------------------------------------------+---------+-------+
|standardized_dept_name  |service_request_type                        |days_late|n_cases|
+------------------------+--------------------------------------------+---------+-------+
|City Council            |Request for Research/Information            |null     |5      |
|Trans & Cap Improvements|Engineering Design                          |-1413.8  |1      |
|Trans & Cap Improvements|Signal Timing Modification By Engineer      |-1352.0  |22     |
|Animal Care Services    |Stray Animal                                |-998.8   |27346  |
|Parks & Recreation      |Major Park Improvement Install              |-278.3   |271    |
|Trans & Cap Improvements|Sidewalk Cost Sharing Program               |-177.8   |131    |
|DSD/Code Enforcement    |Multi Tenant Exterior                       |-135.7   |84     |
|DSD/Code Enforcement    |CPS Energy Towers                           |-129.8   |511    |
|DSD/Code 

In [28]:
(
    df.filter(col("service_request_type") == "Stray Animal")
    .select("case_opened_date", "case_due_date", "case_closed_date")
    .withColumn("days_to_resolve", datediff("case_due_date", "case_opened_date"))
    .show()
)

+-------------------+-------------------+-------------------+---------------+
|   case_opened_date|      case_due_date|   case_closed_date|days_to_resolve|
+-------------------+-------------------+-------------------+---------------+
|2018-01-01 00:42:00|2020-09-26 00:42:00|2018-01-01 12:29:00|            999|
|2018-01-01 08:39:00|2020-09-26 08:39:00|2018-01-01 12:30:00|            999|
|2018-01-01 09:51:00|2020-09-26 09:51:00|2018-01-01 11:53:00|            999|
|2018-01-01 10:39:00|2020-09-26 10:39:00|2018-01-01 12:29:00|            999|
|2018-01-01 10:44:00|2020-09-26 10:44:00|2018-01-01 12:43:00|            999|
|2018-01-01 10:52:00|2020-09-26 10:52:00|2018-01-01 11:53:00|            999|
|2018-01-01 11:11:00|2020-09-26 11:11:00|2018-01-01 11:53:00|            999|
|2018-01-01 11:46:00|2020-09-26 11:46:00|2018-01-01 12:26:00|            999|
|2018-01-01 11:58:00|2020-09-26 11:58:00|2018-01-01 12:26:00|            999|
|2018-01-01 12:28:00|2020-09-26 12:28:00|2018-01-01 12:44:00|   

### Handling Dates
Quick Recap: getting data from spark dataframes:

.show(n): prints the first n rows. Doesn't produce a value that can be used later

.first: gives us the first row object

.head(n): gives us a list of the first n row objects

.collect(): turns all the rows into a list of row objects be careful here

In [29]:
df.select(max('case_opened_date'), max("case_closed_date")).collect()

[Row(max(case_opened_date)=datetime.datetime(2018, 8, 8, 10, 38), max(case_closed_date)=datetime.datetime(2018, 8, 8, 10, 38))]

In [30]:
max_date = df.select(max('case_opened_date'), max('case_closed_date')).first()[0]
max_date

datetime.datetime(2018, 8, 8, 10, 38)

In [31]:
max_date = max_date.strftime('%Y-%m-%d %H:%M:%S')
max_date

'2018-08-08 10:38:00'

In [32]:
df = (
    df.withColumn('case_age', datediff(lit(max_date), 'case_opened_date'))
    .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
    .drop('case_age', 'days_to_closed')
)

In [33]:
#Sidebar: Python Code Formatting
# Indentation conveys nesting

# autoformatting tools remove the discussion

df = (df.withColumn('case_age', datediff(lit(max_date), 'case_opened_date')).withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date')).withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age'))).drop('case_age', 'days_to_closed'))


In [34]:
df = (
    df.withColumn("case_age", datediff(lit(max_date), "case_opened_date"))
    .withColumn("days_to_closed", datediff("case_closed_date", "case_opened_date"))
    .withColumn(
        "case_lifetime",
        when(col("case_closed"), col("days_to_closed")).otherwise(col("case_age")),
    )
    .drop("case_age", "days_to_closed")
)

In [35]:
(df.withColumn('case_age', datediff(lit(max_date), 'case_opened_date'))
 .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
 .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
 .drop('case_age', 'days_to_closed'))

DataFrame[source_id: string, dept_division: string, case_id: int, case_opened_date: timestamp, case_closed_date: timestamp, case_due_date: timestamp, case_late: boolean, num_days_late: double, case_closed: boolean, service_request_type: string, SLA_days: double, case_status: string, request_address: string, council_district: string, zipcode: string, case_lifetime: int, dept_name: string, standardized_dept_name: string, dept_subject_to_SLA: string, source_username: string]

<!-- Install black -- the python code formatter

 python -m pip install black
Create an alias

 alias fmt-clipboard='pbpaste | black -q - | pbcopy'
Put this line in your .zshrc or .bash_profile

 echo $0
Will tell you what shell you are using

Open a new terminal

Copy the code you want to format

Run the fmt-clipboard command

Paste the formatted code

To format a .py file

black script.py
black-nb -->

In [36]:
#Install black -- the python code formatter

 #python -m pip install black
#Create an alias

 #alias fmt-clipboard='pbpaste | black -q - | pbcopy'
#Put this line in your .zshrc or .bash_profile

 #echo $0
#Will tell you what shell you are using

#Open a new terminal

#Copy the code you want to format

#Run the fmt-clipboard command

#Paste the formatted code

#To format a .py file

#black script.py
#black-nb